In [8]:
#!/usr/bin/env python3
import numpy as np
import gym
import os
from pathlib import Path

import matplotlib.pyplot as plt
import time
from IPython import display
%matplotlib notebook

from stable_baselines.common.cmd_util import mujoco_arg_parser
from stable_baselines import bench, logger
from stable_baselines.common import set_global_seeds
from stable_baselines.common.vec_env.vec_normalize import VecNormalize
from stable_baselines.ppo2 import PPO2
from stable_baselines.ppo1 import PPO1
from stable_baselines.common.policies import MlpPolicy, MlpLstmPolicy
from stable_baselines.common.vec_env.dummy_vec_env import DummyVecEnv
from stable_baselines.common.vec_env.subproc_vec_env import SubprocVecEnv
from stable_baselines.results_plotter import load_results, ts2xy

home = str(Path.home())

def train(env_id, num_timesteps, seed):
    """
    Train PPO2 model for Mujoco environment, for testing purposes
    :param env_id: (str) the environment id string
    :param num_timesteps: (int) the number of timesteps to run
    :param seed: (int) Used to seed the random generator.
    """
    def make_env(_env_id):
        env_out = gym.make(_env_id)
        env_out = bench.Monitor(env_out, log_dir, allow_early_resets=True)
        return env_out
    
    n_cpu = 8#multiprocessing.cpu_count()
    #env = SubprocVecEnv([lambda: make_env(env_id) for i in range(n_cpu)])

    env = DummyVecEnv([lambda: make_env(env_id)])
    #env = VecNormalize(env)

    def callback(_locals, _globals):
        global n_steps, best_mean_reward
        print("Step:", n_steps)

        if (n_steps + 1) % 100 == 0:
            _locals['self'].save("model_{}".format(env_id))
        n_steps += 1
        return False

    set_global_seeds(seed)
    policy = MlpPolicy
    model = PPO1(policy=policy, env=env, verbose=1, tensorboard_log=log_dir)
    model.learn(total_timesteps=num_timesteps, callback=callback)
    model.save("model_{}".format(env_id))

    return model, env

In [9]:
env_id='CartPole-v1'
num_timesteps=20000000
seed=343
best_mean_reward, n_steps = -np.inf, 0

base_dir = home + '/ppo_logs'
prev = [f for f in os.listdir(base_dir) if env_id in f]
log_dir = base_dir + '/{}-{}'.format(env_id, len(prev))
os.makedirs(log_dir, exist_ok=True)

print('Logging to {}'.format(log_dir))

logger.configure()
model, env = train(env_id, num_timesteps, seed)

Logging to /home/ubuntu/ppo_logs/CartPole-v1-2
Logging to /tmp/openai-2018-12-02-20-14-22-023037
Step: 0
********** Iteration 0 ************
Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
     -0.00221 |      -0.00693 |     103.78012 |      9.82e-05 |       0.69305
     -0.01107 |      -0.00692 |     102.36276 |       0.00103 |       0.69214
     -0.01956 |      -0.00690 |     100.92700 |       0.00368 |       0.68954
     -0.02522 |      -0.00685 |      99.28327 |       0.00821 |       0.68513
Evaluating losses...
     -0.02643 |      -0.00682 |      98.18385 |       0.01164 |       0.68183
-----------------------------------
| EpLenMean       | 27.1          |
| EpRewMean       | 27.1          |
| EpThisIter      | 9             |
| EpisodesSoFar   | 9             |
| TimeElapsed     | 0.778         |
| TimestepsSoFar  | 256           |
| ev_tdlam_before | 0.000964      |
| loss_ent        | 0.6818348     |
| loss_kl         | 0.0116381645

Step: 8
********** Iteration 8 ************
Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
      0.00054 |      -0.00590 |     120.70531 |       0.00061 |       0.58961
     -0.00494 |      -0.00591 |     112.07675 |       0.00291 |       0.59109
     -0.00661 |      -0.00593 |     103.75373 |       0.00396 |       0.59348
     -0.00916 |      -0.00595 |      96.83098 |       0.00454 |       0.59502
Evaluating losses...
     -0.00964 |      -0.00596 |      92.51924 |       0.00531 |       0.59583
----------------------------------
| EpLenMean       | 44.9         |
| EpRewMean       | 44.9         |
| EpThisIter      | 2            |
| EpisodesSoFar   | 50           |
| TimeElapsed     | 4.28         |
| TimestepsSoFar  | 2594         |
| ev_tdlam_before | 0.0995       |
| loss_ent        | 0.595829     |
| loss_kl         | 0.0053133285 |
| loss_pol_entpen | -0.00595829  |
| loss_pol_surr   | -0.00964312  |
| loss_vf_loss    | 92.51924     

Step: 16
********** Iteration 16 ************
Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
    -8.95e-05 |      -0.00582 |     117.74938 |       0.00031 |       0.58205
     -0.00946 |      -0.00575 |     115.13451 |       0.00266 |       0.57517
     -0.00977 |      -0.00568 |     112.27246 |       0.00637 |       0.56847
     -0.00947 |      -0.00566 |     109.66964 |       0.00736 |       0.56605
Evaluating losses...
     -0.01035 |      -0.00567 |     107.64496 |       0.00644 |       0.56693
-----------------------------------
| EpLenMean       | 68.5          |
| EpRewMean       | 68.5          |
| EpThisIter      | 2             |
| EpisodesSoFar   | 63            |
| TimeElapsed     | 7.79          |
| TimestepsSoFar  | 5599          |
| ev_tdlam_before | 0.019         |
| loss_ent        | 0.5669255     |
| loss_kl         | 0.0064437045  |
| loss_pol_entpen | -0.0056692553 |
| loss_pol_surr   | -0.010354795  |
| loss_vf_loss    |

Step: 24
********** Iteration 24 ************
Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
    -6.68e-05 |      -0.00557 |     142.90269 |       0.00012 |       0.55696
     -0.01066 |      -0.00547 |     134.71417 |       0.00432 |       0.54717
     -0.00777 |      -0.00540 |     131.29736 |       0.00937 |       0.54029
     -0.01238 |      -0.00544 |     131.80032 |       0.00437 |       0.54438
Evaluating losses...
     -0.01102 |      -0.00549 |     128.94067 |       0.00145 |       0.54905
----------------------------------
| EpLenMean       | 87.3         |
| EpRewMean       | 87.3         |
| EpThisIter      | 3            |
| EpisodesSoFar   | 73           |
| TimeElapsed     | 11.3         |
| TimestepsSoFar  | 8406         |
| ev_tdlam_before | 0.111        |
| loss_ent        | 0.5490523    |
| loss_kl         | 0.0014500191 |
| loss_pol_entpen | -0.005490523 |
| loss_pol_surr   | -0.011015154 |
| loss_vf_loss    | 128.94067  

Step: 32
********** Iteration 32 ************
Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
      0.00204 |      -0.00601 |      58.93188 |       0.00019 |       0.60054
     -0.00081 |      -0.00605 |      56.20024 |       0.00070 |       0.60504
     -0.00059 |      -0.00608 |      54.44357 |       0.00148 |       0.60822
     -0.00302 |      -0.00612 |      49.62776 |       0.00187 |       0.61158
Evaluating losses...
     -0.00422 |      -0.00614 |      46.98482 |       0.00217 |       0.61429
----------------------------------
| EpLenMean       | 94           |
| EpRewMean       | 94           |
| EpThisIter      | 1            |
| EpisodesSoFar   | 88           |
| TimeElapsed     | 14.8         |
| TimestepsSoFar  | 10994        |
| ev_tdlam_before | 0.849        |
| loss_ent        | 0.61428624   |
| loss_kl         | 0.0021662754 |
| loss_pol_entpen | -0.006142862 |
| loss_pol_surr   | -0.004221812 |
| loss_vf_loss    | 46.98482   

...

Step: 656
********** Iteration 656 ************
Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
     9.85e-05 |      -0.00578 |     122.99979 |      6.96e-06 |       0.57816
     -0.00055 |      -0.00574 |     123.20426 |       0.00024 |       0.57371
     -0.00400 |      -0.00569 |     122.61702 |       0.00115 |       0.56875
     -0.00736 |      -0.00565 |     121.40935 |       0.00290 |       0.56459
Evaluating losses...
     -0.00892 |      -0.00563 |     120.11848 |       0.00437 |       0.56340
-----------------------------------
| EpLenMean       | 361           |
| EpRewMean       | 361           |
| EpThisIter      | 1             |
| EpisodesSoFar   | 778           |
| TimeElapsed     | 288           |
| TimestepsSoFar  | 309255        |
| ev_tdlam_before | 0.083         |
| loss_ent        | 0.56340057    |
| loss_kl         | 0.0043706037  |
| loss_pol_entpen | -0.0056340056 |
| loss_pol_surr   | -0.0089244135 |
| loss_vf_loss   

Step: 664
********** Iteration 664 ************
Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
      0.00064 |      -0.00551 |     132.10390 |      3.47e-05 |       0.55057
      0.00043 |      -0.00552 |     132.03540 |      9.73e-05 |       0.55214
      0.00029 |      -0.00553 |     131.44765 |      7.81e-05 |       0.55282
    -9.58e-05 |      -0.00553 |     130.73955 |      4.36e-05 |       0.55251
Evaluating losses...
     -0.00035 |      -0.00552 |     130.10962 |      5.19e-05 |       0.55219
------------------------------------
| EpLenMean       | 368            |
| EpRewMean       | 368            |
| EpThisIter      | 1              |
| EpisodesSoFar   | 786            |
| TimeElapsed     | 292            |
| TimestepsSoFar  | 313751         |
| ev_tdlam_before | -0.0207        |
| loss_ent        | 0.55219066     |
| loss_kl         | 5.190629e-05   |
| loss_pol_entpen | -0.005521907   |
| loss_pol_surr   | -0.00035412982 |
| los

Step: 672
********** Iteration 672 ************
Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
     -0.00013 |      -0.00539 |     204.43823 |      2.42e-06 |       0.53870
     -0.00041 |      -0.00540 |     204.26947 |       0.00014 |       0.54005
     -0.00121 |      -0.00543 |     203.93353 |       0.00048 |       0.54256
     -0.00207 |      -0.00547 |     203.49571 |       0.00111 |       0.54662
Evaluating losses...
     -0.00232 |      -0.00548 |     203.26657 |       0.00171 |       0.54848
-----------------------------------
| EpLenMean       | 375           |
| EpRewMean       | 375           |
| EpThisIter      | 1             |
| EpisodesSoFar   | 794           |
| TimeElapsed     | 295           |
| TimestepsSoFar  | 318631        |
| ev_tdlam_before | -0.0106       |
| loss_ent        | 0.5484792     |
| loss_kl         | 0.0017108876  |
| loss_pol_entpen | -0.0054847915 |
| loss_pol_surr   | -0.0023181867 |
| loss_vf_loss   

Step: 680
********** Iteration 680 ************
Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
     -0.00015 |      -0.00583 |     219.25169 |      7.62e-05 |       0.58317
     -0.00513 |      -0.00573 |     218.21564 |       0.00090 |       0.57269
     -0.00975 |      -0.00557 |     216.83353 |       0.00341 |       0.55740
     -0.01138 |      -0.00543 |     215.20746 |       0.00727 |       0.54339
Evaluating losses...
     -0.01156 |      -0.00538 |     214.55441 |       0.00929 |       0.53757
----------------------------------
| EpLenMean       | 364          |
| EpRewMean       | 364          |
| EpThisIter      | 1            |
| EpisodesSoFar   | 803          |
| TimeElapsed     | 299          |
| TimestepsSoFar  | 321933       |
| ev_tdlam_before | 0.00495      |
| loss_ent        | 0.53757364   |
| loss_kl         | 0.009288194  |
| loss_pol_entpen | -0.005375736 |
| loss_pol_surr   | -0.011559766 |
| loss_vf_loss    | 214.55441

KeyboardInterrupt: 